In [1]:
from file_processors import histogram_processor
from feature_extractors import ae_extractor, pca_extractor
from feature_embedders import tsne_embedder
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox
import glob

In [2]:
# Define file paths
data_path = "../datasets"
encoders_path = "../encoders"
properties_file = f"{data_path}/properties-input-Full.csv"
ae_model_2d = f"{encoders_path}/XAE-ENCODER-2D-CNN-cn[32, 16]-fc[192, 48]-neck12-lr0.001-b1024-reg0.0-botreg0.1-kernhe_normal-xae-rs505.h5"
ae_model_3d = f"{encoders_path}/XAE-ENCODER-3D-FC-fc[1536, 384, 96]-neck24-lr0.001-b1024-reg0.1-xae24.h5"

# Widgets
histogram_representation_widget = widgets.Dropdown(
    options=["2D", "3D"],
    value="2D",
    description="Histogram Representation:",
    disabled=False,
    style={'description_width': 'initial'}
)

feature_extractor_widget = widgets.Dropdown(
    options=["PCA", "AE"],
    value="PCA",
    description="Feature Extractor:",
    disabled=False,
    style={'description_width': 'initial'}
)

n_pca_widget = widgets.IntSlider(
    value=2, min=2, max=100, step=1, 
    description="PCA Components:", 
    disabled=False, continuous_update=False, 
    readout=True, readout_format='d',
    style={'description_width': 'initial'}
)

# Container for n_pca_widget
n_pca_container = HBox([n_pca_widget])
n_pca_container.layout.display = 'none'  # Initially hide the container

# Event handler for feature_extractor_widget
def on_feature_extractor_change(change):
    if change['new'] == 'PCA':
        n_pca_container.layout.display = 'block'  # Show the container
    else:
        n_pca_container.layout.display = 'none'   # Hide the container

# Event handler for histogram_representation_widget
def on_histogram_representation_change(change):
    if change['new'] == '2D':
        n_pca_widget.value = 15  # Set default value for 2D
    elif change['new'] == '3D':
        n_pca_widget.value = 22  # Set default value for 3D

# Observe changes in widgets
feature_extractor_widget.observe(on_feature_extractor_change, names='value')
histogram_representation_widget.observe(on_histogram_representation_change, names='value')

# Display the widgets
display(HBox([histogram_representation_widget, feature_extractor_widget]))
display(n_pca_container)  # Display the container

# Initial calls to set the correct states
on_feature_extractor_change({'new': feature_extractor_widget.value})
on_histogram_representation_change({'new': histogram_representation_widget.value})


# # Select 2D vs 3D histogram representations and PCA vs AE feature extractors
# histogram_representation_widget = widgets.Dropdown(options=["2D", "3D"], value="2D", description="Histogram Representation:", disabled=False, style = {'description_width': 'initial'})
# feature_extractor_widget = widgets.Dropdown(options=["PCA", "AE"], value="PCA", description="Feature Extractor:", disabled=False, style = {'description_width': 'initial'})
# n_pca_widget = widgets.IntSlider(value=2, min=2, max=100, step=1, description="PCA Components:", disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
# display(HBox([histogram_representation_widget, feature_extractor_widget]))

In [5]:
# Display widget selections
print("Selected Histogram Representation Type: ", histogram_representation_widget.value)
print("Selected Feature Extractor: ", feature_extractor_widget.value)
if feature_extractor_widget.value == "PCA":
    print("Selected PCA Components: ", n_pca_widget.value)

# Select appropriate histogram file
if feature_extractor_widget.value == "PCA":
    if histogram_representation_widget.value == "2D":
        histogram_file = glob.glob(f"{data_path}/hist{histogram_representation_widget.value}*none*")[0]
    elif histogram_representation_widget.value == "3D":
        histogram_file = glob.glob(f"{data_path}/hist{histogram_representation_widget.value}*none*")[0]
elif feature_extractor_widget.value == "AE":
    if histogram_representation_widget.value == "2D":
        histogram_file = glob.glob(f"{data_path}/hist{histogram_representation_widget.value}*minmax*")[0]
    elif histogram_representation_widget.value == "3D":
        histogram_file = glob.glob(f"{data_path}/hist{histogram_representation_widget.value}*minmax*")[0]
print("Selected Histogram Representation File: ", histogram_file)

# Process histogram file and properties table
histograms, df_properties = histogram_processor.histogram_processor(histogram_file,properties_file, feature_extractor_widget.value)

# Extract features from histograms
if feature_extractor_widget.value == "PCA":
        features = pca_extractor.pca_extractor(histograms, int(n_pca_widget.value))
elif feature_extractor_widget.value == "AE":
    if histogram_representation_widget.value == "2D":
        features = ae_extractor.ae_extractor(histograms, ae_model_2d)
    elif histogram_representation_widget.value == "3D":
        features = ae_extractor.ae_extractor(histograms, ae_model_3d)


Selected Histogram Representation Type:  2D
Selected Feature Extractor:  AE
Selected Histogram Representation File:  ../datasets/hist2D-Full-nE16-nt24-normminmax.pkl
Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-01-24 09:51:48.056850: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2984/2984 [==============================] - 8s 2ms/step
